# Práctica 2: Transformaciones geométricas

### Participantes:
- Gerardo León Quintana
- Susana Suárez Mendoza

In [1]:
import cv2 as cv
import numpy as np
import time

### 1a. Desarrollar una aplicación que lleve a cabo transformaciones de la imagen en tiempo real a través de una interfaz basada en trackbars o equivalente.

- Hacer traslaciones. Es necesario indicar la magnitud de la traslación en X y en Y.
- Hacer rotaciones. Es necesario indicar el centro de giro y ángulo de giro.
- Hacer escalados uniformes y no uniformes. Es necesario indicar los factores de escala.

In [3]:
img = cv.imread('./images/img_cuadro.jpg', cv.IMREAD_COLOR)
if img is None:
    raise FileNotFoundError("La imagen no se encontró en la ruta especificada.")

def nothing(x):
    pass

height, width = img.shape[:2]
size = (width, height)


cv.namedWindow('image')

cv.createTrackbar('X', 'image', 0, width, nothing)
cv.createTrackbar('Y', 'image', 0, height, nothing)


cv.createTrackbar('Angulo', 'image', 0, 360, nothing)


cv.createTrackbar('Escala', 'image', 50, 100, nothing)
cv.createTrackbar('Escala X', 'image', 50, 100, nothing)
cv.createTrackbar('Escala Y', 'image', 50, 100, nothing)

mode = 'Traslación'

center = (width // 2, height // 2)


show_text = False
text_display_time = 2  # segundos
text_start_time = 0

def get_center(event, x, y, flags, param):
    global center, show_text, text_start_time
    if event == cv.EVENT_LBUTTONDOWN:
        center = (x, y)
    elif event == cv.EVENT_LBUTTONUP:
        return center
        


cv.setMouseCallback('image', get_center)

cv.imshow('image', img)

while True:

    key = cv.waitKey(1) & 0xFF

    if key == ord('m'):
        if mode == 'Traslación':
            mode = 'Rotación'
        elif mode == 'Rotación':
            mode = 'Escalado Uniforme'
        elif mode == 'Escalado Uniforme':
            mode = 'Escalado No Uniforme'
        elif mode == 'Escalado No Uniforme':
            mode = 'Traslación'  

        show_text = True
        text_start_time = time.time()

 
    img_display = img.copy()


    tx = cv.getTrackbarPos('X', 'image')
    ty = cv.getTrackbarPos('Y', 'image')
    angle = cv.getTrackbarPos('Angulo', 'image')
    escala = cv.getTrackbarPos('Escala', 'image')
    escala_x = cv.getTrackbarPos('Escala X', 'image')
    escala_y = cv.getTrackbarPos('Escala Y', 'image')

    if mode == 'Traslación':

        T = np.float32([[1, 0, tx], [0, 1, ty]])
        img_display = cv.warpAffine(img_display, T, size)
        texto = f'Modo: Traslacion'

    elif mode == 'Rotación':

        R = cv.getRotationMatrix2D(center, angle, 1)
        img_display = cv.warpAffine(img_display, R, size)
        texto = f'Modo: Rotacion'

    elif mode == 'Escalado Uniforme':

        scale_factor = escala / 50.0  

        S = cv.getRotationMatrix2D(center, 0, scale_factor)
        img_display = cv.warpAffine(img_display, S, size)
        texto = f'Modo: Escalado Uniforme'

    elif mode == 'Escalado No Uniforme':

        scale_x_factor = escala_x / 50.0  
        scale_y_factor = escala_y / 50.0

        S = np.float32([[scale_x_factor, 0, (1 - scale_x_factor) * center[0]],
                        [0, scale_y_factor, (1 - scale_y_factor) * center[1]]])
        img_display = cv.warpAffine(img_display, S, size)
        texto = f'Modo: Escalado No Uniforme'


    current_time = time.time()
    if show_text:
        elapsed_time = current_time - text_start_time
        if elapsed_time < text_display_time:

            cv.rectangle(img_display, (0, 0), (500, 30), (0, 0, 0), cv.FILLED)

            cv.putText(img_display, texto, (10, 20), cv.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2, cv.LINE_AA)
        else:

            show_text = False

    cv.imshow('image', img_display)

    if key == 27:
        break

cv.destroyAllWindows()


### 1b. Dada una imagen trazar una ventana de proyección y proyectar la imagen.

In [6]:
img_orig = cv.imread('./images/img_cuadro.jpg', cv.IMREAD_COLOR)
img = img_orig.copy()
points = []
actions = []
w,h = img.shape[1], img.shape[0]
size = (w,h)
pts1 = np.float32([[0,0], [0,h], [w,h], [w,0]])

def save_point(event, x, y, flags, param):
    global points
    if event == cv.EVENT_LBUTTONDOWN:
        points.append([x, y])
    elif event == cv.EVENT_LBUTTONUP:
        cv.circle(img, (x, y), 5, (0, 0, 255), -1)
        actions.append(img.copy())
        cv.imshow('image', img)

cv.namedWindow('image')
cv.setMouseCallback('image', save_point)
cv.imshow('image', img)

while True:
    key = cv.waitKey(1) & 0xFF

    if key == ord('q'):
        break
    if len(points) == 4:
        pts2 = np.float32([points[0], points[1], points[2], points[3]])
        
        Tp = cv.getPerspectiveTransform(pts1, pts2)
        img_p = cv.warpPerspective(img_orig, Tp, size)

        pts = pts2.astype(np.int32)
        pts = pts.reshape((-1,1,2))
        cv.polylines(img, [pts], True, (0, 255, 0), 2)
        cv.imshow('image', img_p)

        
    elif key == ord('d'):
        points = []
        actions = []
        cv.imshow('image', img)
    elif key == ord('r') and len(actions) != 0:
        points.pop()
        actions.pop()
        img = actions[-1].copy()
        cv.imshow('image', img)
cv.destroyAllWindows()

### 1c. Desarrollar una aplicación que lleve a cabo distorsiones de la lente. Para ello los coeficientes de distorsión deben gobernarse a través de una interfaz

In [2]:
def apply_distortion(image, k1, k2):
    h, w = image.shape[:2]
    distCoeff = np.zeros((4,1), np.float64)
    distCoeff[0,0] = k1
    distCoeff[1,0] = k2

    cam = np.eye(3, dtype=np.float32)
    cam[0,2] = w/2.0 
    cam[1,2] = h/2.0
    cam[0,0] = 10.
    cam[1,1] = 10.

    distorted_img = cv.undistort(image, cam, distCoeff)

    return distorted_img

In [5]:
def update_image(val):
    global image
    k1 = (cv.getTrackbarPos('K1', 'DistLente') - 100) / 100000.0
    k2 = (cv.getTrackbarPos('K2', 'DistLente') - 100) / 100000.0

    if k1 > 0 and k2 > 0:
        distorted_img = apply_distortion(image, k1, k2)
        return distorted_img
    elif k1 < 0 and k2 < 0:
        distorted_img = apply_distortion(image, k1, k2)
        return distorted_img
    else:
        return image

def nothing(x):
    pass

In [6]:
img = cv.imread('images/img_cuadro.jpg', cv.IMREAD_COLOR)
image = img.copy()

cv.namedWindow('DistLente')
cv.createTrackbar('K1', 'DistLente', 100, 200, nothing)
cv.createTrackbar('K2', 'DistLente', 100, 200, nothing)

update_image(0)

while True:
    key = cv.waitKey(1) & 0xFF
    
    if key == 27:
        break
    elif key == ord('d'):
        cv.imshow('DistLente', img)
        image = img.copy()

    disorted_img = update_image(image)
    cv.imshow('DistLente', disorted_img)

cv.destroyAllWindows()

### Optativo

- Marcar el punto de giro con el ratón.
- Trasladar la imagen arrastrándolo con el ratón y visualizarlo en tiempo real.
- Hacer la parte obligatoria sobre vídeo en lugar de sobre imagen.
- Dada una imagen seleccionar tres puntos de la imagen original y tres puntos en  una imagen destino y realizar la transformación afín.

- Calcular la imagen especular a partir de una imagen.
- Tratar una recta que será el eje de reflexión y “reflejar” la imagen. 
- Otras aportacione

- Marcar el punto de giro con el ratón.

Este apartado optativo ya está implementado dentro del apartado 1a de la parte obligatoria de la práctica, esto lo hemos implementado mediante la creación de una función denominada **get_center**, dicha función nos permite extraer la coordenada de la matriz en la que clickemos.

- Trasladar la imagen arrastrándolo con el ratón y visualizarlo en tiempo real

In [14]:
img = cv.imread('./images/img_cuadro.jpg', cv.IMREAD_COLOR)
if img is None:
    raise FileNotFoundError("La imagen no se encontró en la ruta especificada.")

def nothing(x):
    pass

height, width = img.shape[:2]
size = (width, height)


cv.namedWindow('image')

cv.createTrackbar('Angulo', 'image', 0, 360, nothing)

cv.createTrackbar('Escala', 'image', 50, 100, nothing)
cv.createTrackbar('Escala X', 'image', 50, 100, nothing)
cv.createTrackbar('Escala Y', 'image', 50, 100, nothing)

mode = 'Traslación'

center = (width // 2, height // 2)
dragging = False
prev_mouse_pos = center


show_text = False
text_display_time = 2  # segundos
text_start_time = 0

def get_center(event, x, y, flags, param):
    global center, show_text, text_start_time, dragging, prev_mouse_pos
    if event == cv.EVENT_LBUTTONDOWN:
        center = (x, y)
        if mode == 'Traslación':
            dragging = True


    elif event == cv.EVENT_MOUSEMOVE and dragging:
        dx = x - prev_mouse_pos[0]
        dy = y - prev_mouse_pos[1]
        center = (center[0] + dx, center[1] + dy)
        prev_mouse_pos = (x, y)

    elif event == cv.EVENT_LBUTTONUP:
        dragging = False
        return center
        


cv.setMouseCallback('image', get_center)

cv.imshow('image', img)

while True:

    key = cv.waitKey(1) & 0xFF

    if key == ord('m'):
        if mode == 'Traslación':
            mode = 'Rotación'
        elif mode == 'Rotación':
            mode = 'Escalado Uniforme'
        elif mode == 'Escalado Uniforme':
            mode = 'Escalado No Uniforme'
        elif mode == 'Escalado No Uniforme':
            mode = 'Traslación'  

        show_text = True
        text_start_time = time.time()

 
    img_display = img.copy()

    angle = cv.getTrackbarPos('Angulo', 'image')
    escala = cv.getTrackbarPos('Escala', 'image')
    escala_x = cv.getTrackbarPos('Escala X', 'image')
    escala_y = cv.getTrackbarPos('Escala Y', 'image')

    if mode == 'Traslación':

        T = np.float32([[1, 0, center[0] - width // 2],
                        [0, 1, center[1] - height // 2]])
        img_display = cv.warpAffine(img_display, T, size)
        texto = f'Modo: Traslacion'

    elif mode == 'Rotación':

        R = cv.getRotationMatrix2D(center, angle, 1)
        img_display = cv.warpAffine(img_display, R, size)
        texto = f'Modo: Rotacion'

    elif mode == 'Escalado Uniforme':

        scale_factor = escala / 50.0  

        S = cv.getRotationMatrix2D(center, 0, scale_factor)
        img_display = cv.warpAffine(img_display, S, size)
        texto = f'Modo: Escalado Uniforme'

    elif mode == 'Escalado No Uniforme':

        scale_x_factor = escala_x / 50.0  
        scale_y_factor = escala_y / 50.0

        S = np.float32([[scale_x_factor, 0, (1 - scale_x_factor) * center[0]],
                        [0, scale_y_factor, (1 - scale_y_factor) * center[1]]])
        img_display = cv.warpAffine(img_display, S, size)
        texto = f'Modo: Escalado No Uniforme'


    current_time = time.time()
    if show_text:
        elapsed_time = current_time - text_start_time
        if elapsed_time < text_display_time:

            cv.rectangle(img_display, (0, 0), (500, 30), (0, 0, 0), cv.FILLED)

            cv.putText(img_display, texto, (10, 20), cv.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2, cv.LINE_AA)
        else:

            show_text = False

    cv.imshow('image', img_display)

    if key == 27:
        break

cv.destroyAllWindows()


- Hacer la parte obligatoria sobre vídeo en lugar de sobre imagen.


- Dada una imagen seleccionar tres puntos de la imagen original y tres puntos en  una imagen destino y realizar la transformación afín.

In [2]:
img = cv.imread('./images/img_cuadro.jpg', cv.IMREAD_COLOR)
if img is None:
    raise FileNotFoundError("La imagen no se encontró en la ruta especificada.")

def nothing(x):
    pass

height, width = img.shape[:2]
size = (width, height)


show_text = False
text_display_time = 2  # segundos
text_start_time = 0

points_orig = []
points_dest = []

def select_points(event, x, y, flags, param):
    global points_orig, points_dest
    if event == cv.EVENT_LBUTTONDOWN:
        if len(points_orig) < 3:
            points_orig.append([x, y])
            cv.circle(img, (x, y), 5, (0, 0, 255), -1)
        elif len(points_orig) == 3 and len(points_dest) < 3:
            points_dest.append([x, y])
            cv.circle(img, (x, y), 5, (255, 0, 0), -1)


def apply_affine_transform(img, points_orig, points_dest):
    pts1 = np.float32(points_orig)
    pts2 = np.float32(points_dest)

    M = cv.getAffineTransform(pts1, pts2)

    img_transformed = cv.warpAffine(img, M, size)

    return img_transformed

cv.namedWindow('image')
cv.setMouseCallback('image', select_points)
cv.imshow('image', img)
img_orig = img.copy()
img_copy = img.copy()

while True:

    key = cv.waitKey(1) & 0xFF


    cv.imshow('image', img)

    if key == 27:
        break

    if len(points_orig) == 3 and len(points_dest) == 3:
        img = apply_affine_transform(img_copy, points_orig, points_dest)
        img_copy = img.copy()
        points_orig = []
        points_dest = []
    
    if key == ord('d'):
        img = img_orig.copy()
        points_orig = []
        points_dest = []

cv.destroyAllWindows()


- Calcular la imagen especular a partir de una imagen.

In [4]:
img = cv.imread('./images/img_cuadro.jpg', cv.IMREAD_COLOR)
if img is None:
    raise FileNotFoundError("La imagen no se encontró en la ruta especificada.")

height, width = img.shape[:2]
size = (width, height)

cv.namedWindow('image')
mode = 'Normal'


show_text = False
text_display_time = 2  # segundos
text_start_time = 0

cv.imshow('image', img)

while True:

    key = cv.waitKey(1) & 0xFF

    if key == ord('m'):
        if mode == 'Normal':
            mode = 'Especular horizontal'
        elif mode == 'Especular horizontal':
            mode = 'Especular vertical'
        elif mode == 'Especular vertical':
            mode = 'Especular ambas'
        elif mode == 'Especular ambas':
            mode = 'Normal'  

        show_text = True
        text_start_time = time.time()
    
    if key == ord('d'):
        mode = 'Normal'
        show_text = True
        text_start_time = time.time()

 
    img_display = img.copy()


    if mode == 'Normal':
        img_display = img.copy()
        texto = f'Modo: Imagen original'

    elif mode == 'Especular horizontal':
        img_display = cv.flip(img, 0)
        texto = f'Modo: Especular eje X'

    elif mode == 'Especular vertical':
        img_display = cv.flip(img, 1)
        texto = f'Modo: Especular eje Y'

    elif mode == 'Especular ambas':
        img_display = cv.flip(img, -1)
        texto = f'Modo: Especular ambos ejes'


    current_time = time.time()
    if show_text:
        elapsed_time = current_time - text_start_time
        if elapsed_time < text_display_time:
            cv.rectangle(img_display, (0, 0), (500, 30), (0, 0, 0), cv.FILLED)

            cv.putText(img_display, texto, (10, 20), cv.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2, cv.LINE_AA)
        else:

            show_text = False

    cv.imshow('image', img_display)

    if key == 27:
        break

cv.destroyAllWindows()